# Trabalho 3 - Uso de Técnicas de ML Aplicadas em Trading Sistemático

**Aluno:** Luiz Fernando Rabelo (11796893)

## Bibliotecas Utilizadas

Para a resolução do trabalho, foram utilizadas as bibliotecas numpy, ta (Technical Analysis), yfinance (Yahoo Finance) e sklearn, as quais são importadas abaixo:

In [ ]:
import numpy as np

from ta.momentum import RSIIndicator
from ta.volume import MFIIndicator
from ta.trend import MACD

import yfinance as yf

from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn import metrics
from sklearn.exceptions import ConvergenceWarning

import warnings
warnings.filterwarnings("ignore", category=ConvergenceWarning)

## Ativos Considerados

A fim de compor uma lista de ações a serem testadas, foi priorizada a diversificação do setor de atuação das empresas. Nesse sentido, considerando um período de análise variando de 1 de janeiro de 2003 à 31 de dezembro de 2022, foram escolhidas as seguintes ações:

- MGLU3 (Magazine Luiza);
- VALE3 (Vale);
- OIBR3 (Oi);
- BBAS3 (Banco do Brasil);
- POMO4 (Marcopolo).

Nota: nem todas as ações possuem dados registrados no período supracitado. Nesses casos, foi obtido um histórico de um período máximo das ações.

In [ ]:
# Inicialização de lista de ações e dicionário de dados:
acoes_consideradas = ['MGLU3.SA', 'VALE3.SA', 'OIBR3.SA', 'BBAS3.SA', 'POMO4.SA']
dados_acoes = {acao: {} for acao in acoes_consideradas}

In [ ]:
# Definição do intervalo de consulta:
data_inicio = '2003-01-01'
data_fim = '2022-12-31'

In [ ]:
# Armazenamento dos dados históricos:
for acao in acoes_consideradas:
    historico = yf.download(acao, data_inicio, data_fim, '1d')
    dados_acoes[acao] = historico[['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']]
    # Adequação dos dados ao fechamento ajustado:
    for index, row in dados_acoes[acao].iterrows():
        dados_acoes[acao].at[index, 'High'] = row['High'] / row['Close'] * row['Adj Close']
        dados_acoes[acao].at[index, 'Low'] = row['Low'] / row['Close'] * row['Adj Close']
        dados_acoes[acao].at[index, 'Close'] = row['Adj Close']

## Estratégias de Aprendizado

As ações foram analisadas, individualmente, para definir uma tática de trading sistemático com base na classificação de algoritmos de aprendizado de máquina. Esses algoritmos levaram em consideração diferentes indicadores de entrada:

- Relative Strength Index (momentum);
- Money Flow Index (volume);
- Moving Average Convergence Divergence (tendência).

Para definir uma referência a ser seguida, foram estabelecidas janelas de 11 dias, sobre as quais foram calculados os pontos de mínimo (compra ideal) e de máximo (venda ideal) da ação.

A missão dos algoritmos de ML conduziu-se na tentativa de combinar informações desses indicadores de forma a tomar decisões de compra e venda de uma ação. Durante o período de análise, foram realizados vários treinamentos (em partições de 400 dias, sendo 120 dias para treinamento e 280 para teste).

Os valores dos indicadores foram convertidos para a mesma ordem de grandeza a fim de simplificar o treinamento.

In [ ]:
def obtem_parametros_entrada(dados):
    # Relative Strength Index:
    ind_rsi = RSIIndicator(close=dados['Close'], window=24, fillna=True)
    dados['rsi_r'] = ind_rsi.rsi() / 100.0

    # Money Flow Index:
    ind_mfi = MFIIndicator(high=dados['High'], low=dados['Low'], close=dados['Close'], volume=dados['Volume'], window=24, fillna=True)
    dados['mfi_r'] = ind_mfi.money_flow_index() / 100.0

    # Moving Average Convergence Divergence:
    ind_macd = MACD(close=dados['Close'], window_slow=24, window_fast=12, fillna=True)
    dados['macd'] = ind_macd.macd()

    return dados[['rsi_r', 'mfi_r', 'macd']].reset_index().drop(['Date'], axis=1).to_numpy()

In [ ]:
# Definição de valores das decisões:
COMPRA, MANTEM, VENDE = 1, 0, -1

def obtem_parametros_saida(dados):
    # Criação da coluna "Action" para decisões:
    dados['Action'] = MANTEM

    # Inicialização de variáveis auxiliares:
    periodo_inicio = 0
    tamanho_janela = 11 
    periodo_fim = periodo_inicio + tamanho_janela
    total_dados = dados.shape[0]

    # Definição dos pontos de compra e venda ideais:
    while periodo_fim < total_dados:
        minimo = dados['Close'].iloc[periodo_inicio:periodo_fim].min()
        maximo = dados['Close'].iloc[periodo_inicio:periodo_fim].max()
        for i in range(periodo_inicio, periodo_fim + 1):
            if dados['Close'].iloc[i] == minimo:
                dados.loc[dados.index[i], 'Action'] = MANTEM
                dados.loc[dados.index[i+1], 'Action'] = COMPRA
            elif dados['Close'].iloc[i] == maximo:
                dados.loc[dados.index[i], 'Action'] = MANTEM
                dados.loc[dados.index[i+1], 'Action'] = VENDE
        periodo_inicio = periodo_fim + 1
        periodo_fim = periodo_inicio + tamanho_janela
    
    return dados['Action'].reset_index().drop(['Date'], axis=1).to_numpy()

In [ ]:
def realiza_treinamento(dados_entrada, dados_saida, clf, dataframe):
    # Definição do tamanho de cada partição:
    tamanho_particao = 400

    # Definição da quantidade de dados para treino por partição:
    qtd_treino = 120

    # Cálculo da quantidade de partições (treinos e testes):
    total_particoes = dados_entrada.shape[0] // tamanho_particao

    # Criação da coluna "ML-Action":
    dataframe['ML-Action'] = MANTEM

    # Criação do vetor que conterá as ações finais:
    acoes_finais = []

    # Inicialização de variáveis auxiliares para exibição de acurácia:
    soma_acuracia_treino = soma_acuracia_teste = 0

    # Treino e teste por partição:
    for p in range(total_particoes):
        i = p * tamanho_particao  # índice de início da partição

        # Definição dos dados de treino:
        dados_entrada_treino = dados_entrada[i:i+qtd_treino]
        dados_saida_treino = dados_saida[i+1:i+qtd_treino+1]

        # Definição dos dados de teste:
        dados_entrada_teste = dados_entrada[i+qtd_treino:i+tamanho_particao]
        dados_saida_teste = dados_saida[i+qtd_treino+1:i+tamanho_particao+1]

        # Treino do modelo:
        clf.fit(dados_entrada_treino, dados_saida_treino.ravel());

        # Avaliação dos resultados do conjunto de dados de treinamento:
        dados_preditos_treino = clf.predict(dados_entrada_treino)
        soma_acuracia_treino += metrics.accuracy_score(dados_saida_treino, dados_preditos_treino)

        # Avaliação dos resultados do conjunto de dados de teste:
        dados_preditos_teste = clf.predict(dados_entrada_teste)
        soma_acuracia_teste += metrics.accuracy_score(dados_saida_teste, dados_preditos_teste)

        # Contabilização das decisões:
        acoes_finais = np.append(acoes_finais, dados_preditos_treino)
        acoes_finais = np.append(acoes_finais, dados_preditos_teste) 
    
    # Exibição da média das acurácias em treinos e testes para todas as partições:
    print('Acurácia Média Treino:', round(soma_acuracia_treino / total_particoes, 2))
    print('Acurácia Média Teste:', round(soma_acuracia_teste / total_particoes, 2))
    
    # Adição das decisões ao dataframe:
    acoes_finais = np.append(acoes_finais, [MANTEM] * (dataframe.shape[0] % tamanho_particao))
    dataframe['ML-Action'] = acoes_finais

In [ ]:
def avalia_resultado_trade(dados, investimento_inicial, nome_ativo, nome_algoritmo):
    # Criação da coluna "Saldo":
    dados['Saldo'] = 0

    # Inicialização de variáveis auxiliares:
    saldo = investimento_inicial
    operacao = MANTEM
    total_trades = total_positivos = 0
    media_positivos = media_negativos = 0

    # Registro do valor do saldo:
    for i in range(dados.shape[0]):
        if dados['ML-Action'].iloc[i] == COMPRA and operacao == MANTEM:
            valor_negociado = dados['Close'].iloc[i]
            operacao = COMPRA
        elif dados['ML-Action'].iloc[i] == COMPRA and operacao == VENDE:
            retorno_trade = valor_negociado / dados['Close'].iloc[i] - 1
            if retorno_trade > 0:
                total_positivos += 1
                media_positivos += retorno_trade
            else:
                media_negativos += retorno_trade
            saldo *= (1 + retorno_trade)
            total_trades += 1
            operacao = MANTEM
        elif dados['ML-Action'].iloc[i] == VENDE and operacao == MANTEM:
            valor_negociado = dados['Close'].iloc[i]
            operacao = VENDE
        elif dados['ML-Action'].iloc[i] == VENDE and operacao == COMPRA:
            retorno_trade = dados['Close'].iloc[i] / valor_negociado - 1
            if retorno_trade > 0:
                total_positivos += 1
                media_positivos += retorno_trade
            else:
                media_negativos += retorno_trade
            saldo *= (1 + retorno_trade)
            total_trades += 1
            operacao = MANTEM
        dados.loc[dados.index[i], 'Saldo'] = saldo

    # Impressão de informações sobre as operações realizadas:
    if total_positivos > 0:
        print(f'Trades totais: {total_trades}, Trades positivos: {total_positivos} ({round(total_positivos / total_trades * 100, 2)}%)')
        print(f'Retorno médio trades positivos: {round(media_positivos / total_positivos * 100, 2)}%, Retorno médio trades negativos: {round(media_negativos / (total_trades - total_positivos) * 100, 2) if total_positivos != total_trades else 0}%')
        print(f'Rentabilidade buy and hold: {round((dados["Close"].iloc[-1] / dados["Close"].iloc[0] - 1) * 100, 2)}%, Rentabilidade ML trade: {round((dados["Saldo"].iloc[-1] / dados["Saldo"].iloc[0] - 1) * 100, 2)}%')
    else:
        print('Não foram feitos trades no período')
    
    # Plot da Curva de Capital:
    dados[['Close', 'Saldo']] = dados[['Close', 'Saldo']] / dados[['Close', 'Saldo']].iloc[0] * 100
    dados[['Close', 'Saldo']].iloc[0:].plot(figsize = (15,5), title=f'Curva de Capital {nome_ativo} - {nome_algoritmo}');

## Análises por Ativo

### Magazine Luiza - MGLU3


In [ ]:
dados = dados_acoes['MGLU3.SA'].copy()

#### Obtenção dos Dados de Entrada e Saída

In [ ]:
dados_entrada = obtem_parametros_entrada(dados)
dados_saida = obtem_parametros_saida(dados)

print(dados_entrada.shape, dados_saida.shape)

#### Classificador Multilayer Perceptron

In [ ]:
mlp = MLPClassifier(solver='lbfgs', hidden_layer_sizes=(15, ), max_iter=1000000, random_state=1, activation='tanh')
realiza_treinamento(dados_entrada, dados_saida, mlp, dados)

In [ ]:
avalia_resultado_trade(dados, 100, 'MGLU3', 'Multilayer Perceptron')

#### Classificador Random Forest

In [ ]:
rf = RandomForestClassifier(random_state=1, max_depth=7, n_estimators=5)
realiza_treinamento(dados_entrada, dados_saida, rf, dados)

In [ ]:
avalia_resultado_trade(dados, 100, 'MGLU3', 'Random Forest')

### Vale - VALE3


In [ ]:
dados = dados_acoes['VALE3.SA'].copy()

#### Obtenção dos Dados de Entrada e Saída

In [ ]:
dados_entrada = obtem_parametros_entrada(dados)
dados_saida = obtem_parametros_saida(dados)

print(dados_entrada.shape, dados_saida.shape)

#### Classificador Multilayer Perceptron

In [ ]:
mlp = MLPClassifier(solver='lbfgs', hidden_layer_sizes=(15, ), max_iter=1000000, random_state=1, activation='tanh')
realiza_treinamento(dados_entrada, dados_saida, mlp, dados)

In [ ]:
avalia_resultado_trade(dados, 100, 'VALE3', 'Multilayer Perceptron')

#### Classificador Random Forest

In [ ]:
rf = RandomForestClassifier(random_state=1, max_depth=7, n_estimators=5)
realiza_treinamento(dados_entrada, dados_saida, rf, dados)

In [ ]:
avalia_resultado_trade(dados, 100, 'VALE3', 'Random Forest')

### Oi - OIBR3

In [ ]:
dados = dados_acoes['OIBR3.SA'].copy()

#### Obtenção dos Dados de Entrada e Saída

In [ ]:
dados_entrada = obtem_parametros_entrada(dados)
dados_saida = obtem_parametros_saida(dados)

print(dados_entrada.shape, dados_saida.shape)

#### Classificador Multilayer Perceptron

In [ ]:
mlp = MLPClassifier(solver='lbfgs', hidden_layer_sizes=(15, ), max_iter=1000000, random_state=1, activation='tanh')
realiza_treinamento(dados_entrada, dados_saida, mlp, dados)

In [ ]:
avalia_resultado_trade(dados, 100, 'OIBR3', 'Multilayer Perceptron')

#### Classificador Random Forest

In [ ]:
rf = RandomForestClassifier(random_state=1, max_depth=7, n_estimators=5)
realiza_treinamento(dados_entrada, dados_saida, rf, dados)

In [ ]:
avalia_resultado_trade(dados, 100, 'OIBR3', 'Random Forest')

### Banco do Brasil - BBAS3


In [ ]:
dados = dados_acoes['BBAS3.SA'].copy()

#### Obtenção dos Dados de Entrada e Saída

In [ ]:
dados_entrada = obtem_parametros_entrada(dados)
dados_saida = obtem_parametros_saida(dados)

print(dados_entrada.shape, dados_saida.shape)

#### Classificador Multilayer Perceptron

In [ ]:
mlp = MLPClassifier(solver='lbfgs', hidden_layer_sizes=(15, ), max_iter=1000000, random_state=1, activation='tanh')
realiza_treinamento(dados_entrada, dados_saida, mlp, dados)

In [ ]:
avalia_resultado_trade(dados, 100, 'BBAS3', 'Multilayer Perceptron')

#### Classificador Random Forest

In [ ]:
rf = RandomForestClassifier(random_state=1, max_depth=7, n_estimators=5)
realiza_treinamento(dados_entrada, dados_saida, rf, dados)

In [ ]:
avalia_resultado_trade(dados, 100, 'BBASE', 'Random Forest')

### Marco Polo - POMO4


In [ ]:
dados = dados_acoes['POMO4.SA'].copy()

#### Obtenção dos Dados de Entrada e Saída

In [ ]:
dados_entrada = obtem_parametros_entrada(dados)
dados_saida = obtem_parametros_saida(dados)

print(dados_entrada.shape, dados_saida.shape)

#### Classificador Multilayer Perceptron

In [ ]:
mlp = MLPClassifier(solver='lbfgs', hidden_layer_sizes=(15, ), max_iter=1000000, random_state=1, activation='tanh')
realiza_treinamento(dados_entrada, dados_saida, mlp, dados)

In [ ]:
avalia_resultado_trade(dados, 100, 'POMO4', 'Multilayer Perceptron')

#### Classificador Random Forest

In [ ]:
rf = RandomForestClassifier(random_state=1, max_depth=7, n_estimators=5)
realiza_treinamento(dados_entrada, dados_saida, rf, dados)

In [ ]:
avalia_resultado_trade(dados, 100, 'POMO4', 'Random Forest')